In [1]:
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing, time
data = pd.read_csv('3day.csv',sep='\t', keep_default_na=False, header=None, error_bad_lines=False)

In [2]:
import numpy as np
f_data = pd.DataFrame()
tra_data = pd.DataFrame()
val_data = pd.DataFrame()
tes_data = pd.DataFrame()
u_list, s_u_list, pos_list, neg_list = [],[],[],[]
def fun_avg(user,d):
    neg_ratio = 5
    u_list.append(user)
    pos = d[d[3]==1]
    neg = d[d[3]==0]
    l1 = pos.shape[0]
    l2 = neg.shape[0]
    assert l1+l2 == d.shape[0]
    pos=pos.reset_index(drop=True)
    neg=neg.reset_index(drop=True)
    if l1>=3 and not neg.empty:
        s_u_list.append(user)
        pos_list.append(l1)
        neg_list.append(l2)
#    sample train.validation.test
        if l2 >= neg_ratio*l1:
            s_neg = neg[:neg_ratio*l1]
        else:
            s_neg = neg
        s_data = pd.concat([pos,s_neg],ignore_index=True).sample(frac=1).reset_index(drop=True)
        tra_data = s_data[:int(s_data.shape[0]*0.7)]
        val_data = s_data[int(s_data.shape[0]*0.7):int(s_data.shape[0]*0.8)]
        tes_data = s_data[int(s_data.shape[0]*0.8):]
        return s_data, tra_data, val_data, tes_data

def applyParallel(dfGrouped, func):
    res = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(name,group) for name, group in dfGrouped)
    res = list(filter(None, res))
#     print(res)
    d = [item[0] for item in res]
    t = [item[1] for item in res]
    v = [item[2] for item in res]
    s = [item[3] for item in res]
    return pd.concat(d, ignore_index=True),pd.concat(t, ignore_index=True),pd.concat(v, ignore_index=True),pd.concat(s, ignore_index=True)

t1=time.time()
f_data, tra_data, val_data, tes_data = applyParallel(data.groupby(0),fun_avg)
print(time.time()-t1)
f_data

1691.128473520279


,0,1,2,3,4,5,6
0,100006,1592973545,3293810552_2247539567_1,0,3293810552,0,220432880
1,100006,1592973545,3276299345_2247510078_1,1,3276299345,0,575290460
2,100006,1592973545,3005095618_2650885876_3,0,3005095618,0,76311665
3,100006,1592993789,3203003417_2756651445_1,1,3203003417,1,129601935
4,100006,1592993789,2392054666_2247483687_1,0,2392054666,0,633367160
5,100006,1592993789,3073282833_2650790708_1,0,3073282833,0,76311665
6,100006,1592973545,2396007240_2651986562_1,0,2396007240,0,1272418200
7,100006,1592973545,3019879717_2247490895_1,0,3019879717,0,76311665
8,100006,1592973545,3000504818_2652667535_1,0,3000504818,0,"720965141,1530438340"
9,100006,1592993789,3237377160_2247549548_2,1,3237377160,0,"403918720,114275"


In [3]:
f_max_len=20
user_set = set(tra_data[0])
item_set = set(tra_data[2])
biz_set = set(tra_data[4])
user_set.update([item for sublist in tra_data[6] for item in list(map(lambda x: int(x),sublist.split(',')[:f_max_len]))])
user_map = dict(zip(user_set,range(1,len(user_set)+1)))
item_map = dict(zip(item_set,range(1,len(item_set)+1)))
biz_map = dict(zip(biz_set,range(1,len(biz_set)+1)))

In [4]:
tes_data = tes_data[(tes_data[2].isin(item_set))&(tes_data[4].isin(biz_set))]
tes_data[6] = tes_data[6].map(lambda x: ','.join([y for y in x.split(',')[:f_max_len] if int(y) in user_set]))
tes_data = tes_data[tes_data[6]!='']
val_data = val_data[(val_data[2].isin(item_set))&(val_data[4].isin(biz_set))]
val_data[6] = val_data[6].map(lambda x: ','.join([y for y in x.split(',')[:f_max_len] if int(y) in user_set]))
val_data = val_data[val_data[6]!='']


In [ ]:
max(map(lambda x:len(x),list(user_friends.values())))

In [5]:
tra_data[0] = tra_data[0].map(lambda x: user_map[x])
tra_data[2] = tra_data[2].map(lambda x: item_map[x])
tra_data[4] = tra_data[4].map(lambda x: biz_map[x])
tra_data[6] = tra_data[6].map(lambda x: [user_map[int(y)] for y in x.split(',')[:f_max_len]])
tra_data=tra_data.drop([1], axis=1)
val_data[0] = val_data[0].map(lambda x: user_map[x])
val_data[2] = val_data[2].map(lambda x: item_map[x])
val_data[4] = val_data[4].map(lambda x: biz_map[x])
val_data[6] = val_data[6].map(lambda x: [user_map[int(y)] for y in x.split(',')[:f_max_len]])
val_data=val_data.drop([1], axis=1)
tes_data[0] = tes_data[0].map(lambda x: user_map[x])
tes_data[2] = tes_data[2].map(lambda x: item_map[x])
tes_data[4] = tes_data[4].map(lambda x: biz_map[x])
tes_data[6] = tes_data[6].map(lambda x: [user_map[int(y)] for y in x.split(',')[:f_max_len]])
tes_data=tes_data.drop([1], axis=1)
f_data = pd.concat([tra_data, val_data, tes_data],ignore_index=True)
tra_data

,0,2,3,4,5,6
0,52707,9886,0,148527,0,[124344]
1,52707,258764,1,15034,0,[354883]
2,52707,260527,0,88454,0,[430681]
3,52707,302640,1,81625,1,[331982]
4,52707,128258,0,160659,0,[14428]
5,52707,56511,0,107494,0,[430681]
6,52707,247651,0,7041,0,[262986]
7,52709,44674,0,150053,1,[456526]
8,52709,342092,0,21792,0,[529420]
9,52709,51930,0,76049,0,"[80418, 496205, 151546, 125775, 299982, 10012,..."


In [6]:
from collections import defaultdict
user_packages=dict()
user_items=dict()
user_bizs=dict()
user_friends=dict()
pack_neighbors_b=defaultdict(list)
pack_neighbors_f=defaultdict(list)

t1=time.time()
for u,d in f_data.groupby([0]):
# def fun_avg1(u,d):
#     global pack_neighbors_f
    i = list(d[2])
    b = list(d[4])
    f = list(map(lambda x: x+[0]*(f_max_len-len(x)),d[6]))
    packages = list(zip(i,b,f))
    for index in range(len(packages)):
        for j in range(index+1,len(packages)):
            a = set(packages[index][2]) & set(packages[j][2])-{0}
            if a and packages[index][0] != packages[j][0]:
                pack_neighbors_f[str(u)+'_'+str(packages[index][0])].append((len(a),packages[j]))
                pack_neighbors_f[str(u)+'_'+str(packages[j][0])].append((len(a),packages[index]))
    for item in set(i):
        try:
            pack_neighbors_f[str(u)+'_'+str(item)].sort(key=lambda x:x[0], reverse = True)
            pack_neighbors_f[str(u)+'_'+str(item)] = list(map(lambda x: x[1],pack_neighbors_f[str(u)+'_'+str(item)]))
        except:
            print(pack_neighbors_f[str(u)+'_'+str(packages[index][0])])
print(time.time()-t1)
                
t1=time.time()
for name,d in f_data.groupby([0,4]):
# def fun_avg2(name,d):
#     global pack_neighbors_b
    if d.shape[0]>1:
        i = list(d[2])
        b = list(d[4])
        f = list(map(lambda x: x+[0]*(f_max_len-len(x)),d[6]))
        packages = list(zip(i,b,f))
        for index in range(len(packages)):
            tmp = packages.copy()
            del tmp[index]
#             print(name[0],packages[index][0])
            pack_neighbors_b[str(name[0])+'_'+str(packages[index][0])] = tmp
print(time.time()-t1)

t1=time.time()
for user,d in tra_data.groupby(0):
# def fun_avg3(user,d):
#     global user_packages
    pos = d[(d[3]==1)]
    b_pos = d[(d[5] == 1)]
    i = list(pos[2])
    b = list(pos[4])
    f = list(map(lambda x: x+[0]*(f_max_len-len(x)),pos[6]))
    f_ = list(pos[6])
    b_ = list(b_pos[4])
    user_packages[user]=list(zip(i,b,f))
    user_items[user]=i
    user_bizs[user]=b_
    user_friends[user]=[item for sublist in f_ for item in sublist]
print(time.time()-t1) 

200.41934895515442
285.352374792099
131.22755193710327


In [7]:
sum(list(tra_data[3]))/len(set(tra_data[0]))

4.624464425167268

In [8]:
tra_data=tra_data.sample(frac=1).reset_index(drop=True)
tra_data

,0,2,3,4,5,6
0,517628,123135,1,22700,0,[533792]
1,237527,279662,1,112790,0,[364389]
2,306337,117977,0,11744,0,[248088]
3,286909,307025,0,157503,0,[531033]
4,344566,164803,1,80067,0,"[19430, 257575]"
5,324995,116803,0,147734,1,"[284539, 404382]"
6,438229,270496,0,12746,0,[3482]
7,27373,310576,0,71476,0,"[107543, 287062]"
8,154484,51665,0,120234,0,[484088]
9,516095,50338,0,88834,0,[64685]


In [9]:
import tensorflow as tf
import copy
# val_lable = val_data.pop(3)
# tes_lable = tes_data.pop(3)
# tra_lable = tra_data.pop(3)

def to_list(l):
    res = []
    for i in l:
        res.append(i[0])
        res.append(i[1])
        res.extend(i[2])
    return res

def to_tfrecords1(features,file):
    writer=tf.python_io.TFRecordWriter(path=file)
    size = features.shape[0]
    for i in range(size):
        pack = str(features.iloc[i][0])+'_'+str(features.iloc[i][2])
        u_i = copy.deepcopy(user_items[features.iloc[i][0]])
        u_b = copy.deepcopy(user_bizs[features.iloc[i][0]])
        u_f = copy.deepcopy(user_friends[features.iloc[i][0]])
#         print(features.iloc[i][2],features.iloc[i][4],features.iloc[i][6])
#         print(u_i,u_b,u_f)
        u_p = copy.deepcopy(user_packages[features.iloc[i][0]])
#         print(u_p)
        if features.iloc[i][3] == 1:
            u_i.remove(features.iloc[i][2])
#             u_b.remove(features.iloc[i][4])
            for x in features.iloc[i][6]:
                u_f.remove(x)
#             print('-------',u_i,u_b,u_f)
            for j,x in enumerate(u_p):
                if x[0] == features.iloc[i][2]:
                    del u_p[j]
                    break
#             print('-------------------')
#             print(features.iloc[i])
#             print(u_p)
        example=tf.train.Example(
            features=tf.train.Features(
                feature={
                    "user":tf.train.Feature(int64_list=tf.train.Int64List(value=[features.iloc[i][0]])),
                    "item":tf.train.Feature(int64_list=tf.train.Int64List(value=[features.iloc[i][2]])),
                    "biz":tf.train.Feature(int64_list=tf.train.Int64List(value=[features.iloc[i][4]])),
                    "friends":tf.train.Feature(int64_list=tf.train.Int64List(value=features.iloc[i][6])),
                    "user_items":tf.train.Feature(int64_list=tf.train.Int64List(value=u_i)),
                    "user_bizs":tf.train.Feature(int64_list=tf.train.Int64List(value=u_b)),
                    "user_friends":tf.train.Feature(int64_list=tf.train.Int64List(value=u_f)),
                    "user_packages":tf.train.Feature(int64_list=tf.train.Int64List(value=to_list(u_p[:50]))),
                    "pack_neighbors_b":tf.train.Feature(int64_list=tf.train.Int64List(value=to_list(pack_neighbors_b[pack][:20]))),
                    "pack_neighbors_f":tf.train.Feature(int64_list=tf.train.Int64List(value=to_list(pack_neighbors_f[pack][:20]))),
                    "f_max_len":tf.train.Feature(int64_list=tf.train.Int64List(value=[f_max_len])),
                    "n_users":tf.train.Feature(int64_list=tf.train.Int64List(value=[len(user_map)])),
                    "n_items":tf.train.Feature(int64_list=tf.train.Int64List(value=[len(item_map)])),
                    "n_bizs":tf.train.Feature(int64_list=tf.train.Int64List(value=[len(biz_map)])),
                    "label":tf.train.Feature(int64_list=tf.train.Int64List(value=[features.iloc[i][3]])),
                    "label2":tf.train.Feature(int64_list=tf.train.Int64List(value=[features.iloc[i][5]]))
                }
            )
        )
        writer.write(record=example.SerializeToString())

    writer.close()


def to_tfrecords(features,file):
    writer=tf.python_io.TFRecordWriter(path=file)
    size = features.shape[0]
    for i in range(size):
        pack = str(features.iloc[i][0])+'_'+str(features.iloc[i][2])
#         print(features.iloc[i][5])
        example=tf.train.Example(
            features=tf.train.Features(
                feature={
                    "user":tf.train.Feature(int64_list=tf.train.Int64List(value=[features.iloc[i][0]])),
                    "item":tf.train.Feature(int64_list=tf.train.Int64List(value=[features.iloc[i][2]])),
                    "biz":tf.train.Feature(int64_list=tf.train.Int64List(value=[features.iloc[i][4]])),
                    "friends":tf.train.Feature(int64_list=tf.train.Int64List(value=features.iloc[i][6])),
                    "user_items":tf.train.Feature(int64_list=tf.train.Int64List(value=user_items[features.iloc[i][0]][:50])),
                    "user_bizs":tf.train.Feature(int64_list=tf.train.Int64List(value=user_bizs[features.iloc[i][0]][:50])),
                    "user_friends":tf.train.Feature(int64_list=tf.train.Int64List(value=user_friends[features.iloc[i][0]][:50*f_max_len])),
                    "user_packages":tf.train.Feature(int64_list=tf.train.Int64List(value=to_list(user_packages[features.iloc[i][0]][:50]))),
                    "pack_neighbors_b":tf.train.Feature(int64_list=tf.train.Int64List(value=to_list(pack_neighbors_b[pack][:20]))),
                    "pack_neighbors_f":tf.train.Feature(int64_list=tf.train.Int64List(value=to_list(pack_neighbors_f[pack][:20]))),
                    "f_max_len":tf.train.Feature(int64_list=tf.train.Int64List(value=[f_max_len])),
                    "n_users":tf.train.Feature(int64_list=tf.train.Int64List(value=[len(user_map)])),
                    "n_items":tf.train.Feature(int64_list=tf.train.Int64List(value=[len(item_map)])),
                    "n_bizs":tf.train.Feature(int64_list=tf.train.Int64List(value=[len(biz_map)])),
                    "label":tf.train.Feature(int64_list=tf.train.Int64List(value=[features.iloc[i][3]])),
                    "label2":tf.train.Feature(int64_list=tf.train.Int64List(value=[features.iloc[i][5]]))
                }
            )
        )
        writer.write(record=example.SerializeToString())

    writer.close()

to_tfrecords1(tra_data,'train_f1.tfrecords')
to_tfrecords(val_data,'validation_f1.tfrecords')
to_tfrecords(tes_data,'test_f1.tfrecords')


/home/lc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
print(max(list(map(lambda x: len(x),user_packages.values()))),max(list(map(lambda x: len(x),pack_neighbors_b.values()))),max(list(map(lambda x: len(x),pack_neighbors_f.values()))))
print(len(user_map),len(item_map),len(biz_map))

79 91 220
554237 344087 166465


In [ ]:
tra_data.to_csv('3day_tra.csv',sep='\t',header=0,index=0)
val_data.to_csv('3day_val.csv',sep='\t',header=0,index=0)
tes_data.to_csv('3day_tes.csv',sep='\t',header=0,index=0)